In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import spotipy.util as util
from tqdm.notebook import tqdm

import pickle
from sp_client import Spotify_Client

pp = pprint.PrettyPrinter(indent=4)

In [77]:
#gets URIs of every premade Spotify playlist

uris = set()
offset = 0
more_playlists = True
while more_playlists:
    print(offset)
    retrieved = {x['uri'] for x in sp.user_playlists('spotify', offset = offset)['items']}
    if not retrieved:
        more_playlists = False
    else:
        uris.update(retrieved)
        offset += 50
uris = list(uris)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400


In [78]:
#save 
with open('spotify_playlists.pkl', 'wb') as f:
    pickle.dump(uris, f)

In [27]:
playlist_uri = uris[0]

In [69]:
def get_audio_features(uri):
    data = sp.audio_features(uri)[0]
    del data['type'], data['id'], data['uri'], data['track_href'], data['analysis_url']
    return data

In [73]:
def process_playlist(playlist_uri, processed_songs = dict()):
    playlist_songs = sp.playlist_tracks(playlist_uri)['items']
    for song_dict in tqdm(playlist_songs, leave = False):
        song_uri = song_dict['track']['uri']
        name = song_dict['track']['name']
        artists = ','.join(x['name'] for x in song_dict['track']['artists'])
        if song_uri not in processed_songs: #hasn't already been processed
            song_features = get_audio_features(song_uri)
            song_features['name'] = name
            song_features['artist'] = artists
            processed_songs[song_uri] = song_features
    return processed_songs

In [74]:
def process_all_playlists(playlist_uris, processed_songs = dict()):
    for p_uri in tqdm(playlist_uris):
        processed_songs = process_playlist(p_uri, processed_songs)
    return pd.DataFrame.from_dict(processed_songs, orient = 'index')
    

In [75]:
df = process_all_playlists(uris[:2])

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [76]:
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,name,artist
spotify:track:0ICfYaolLKtzKeRDucQu2u,0.467,0.01310,11,-27.963,1,0.0331,0.9930,0.906000,0.1010,0.0936,140.033,280125,4,Harvest,Pathos Humano
spotify:track:0bF7Po3DHoTqcigWwgd2Xi,0.643,0.00953,5,-24.441,1,0.0500,0.9940,0.914000,0.1000,0.3690,61.048,166667,4,Tiny Elephants,Early Still
spotify:track:1YRE1YKj2UIRdq2zT8PLNY,0.357,0.04300,1,-24.968,1,0.0373,0.9110,0.899000,0.1110,0.0385,79.991,210008,4,Persistent,Permaculti
spotify:track:0RAubp6u1ePTYC6SgXoWcY,0.266,0.00375,4,-34.872,1,0.0388,0.9950,0.887000,0.0916,0.1850,68.105,151500,5,Daydream,IGRAINE
spotify:track:2T9BgAmZQT2dLqllKA5UPj,0.165,0.00847,9,-33.250,1,0.0419,0.9820,0.863000,0.0983,0.0431,71.749,192500,4,Somnolence,Zak Brady
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
spotify:track:5bVXmsElBv3xtPsoLJ1Mj0,0.450,0.65800,3,-7.877,1,0.0887,0.5240,0.000028,0.1300,0.4180,169.670,184000,4,Brand New Colony,Handsome Ghost
spotify:track:7AKebOEDMEZWMqBl4bKoYu,0.514,0.20600,5,-13.578,0,0.0320,0.8350,0.000000,0.1140,0.1270,75.874,212300,4,Chandelier,Kina Grannis
spotify:track:6gf2j6a8uN53XCBcHyxQYZ,0.587,0.62700,0,-7.099,1,0.0265,0.0366,0.095000,0.1530,0.2400,132.030,218506,4,Love Is A Stranger,Hannah Georgas
spotify:track:4ulix0RIyXmuNLgwPaz46S,0.551,0.08760,7,-25.719,1,0.1540,0.7990,0.002430,0.1780,0.2680,135.895,160588,4,Youth,Haux
